In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')

In [ ]:
train_data = train_data.drop(columns=['Name','Cabin'])
train_data['family_member'] = train_data['SibSp'] + train_data['Parch']
train_data = train_data.drop(columns=['SibSp', 'Parch'])

In [ ]:
test_data = test_data.drop(columns=['Name','Cabin'])
test_data['family_member'] = test_data['SibSp'] + test_data['Parch']
test_data = test_data.drop(columns=['SibSp', 'Parch'])

**We can see the wealthier passengers in the higher classes tend to be older, which makes sense. We'll use these average age values to impute based on Pclass for Age.**

In [ ]:
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):
        
        if Pclass == 1 :
            return 37
        elif Pclass == 2:
            return 29
        else:
            return 24
    else:
        return Age

**Now apply that function!**

In [ ]:
train_data['Age'] = train_data[['Age', 'Pclass']].apply(impute_age, axis=1)
test_data['Age'] = test_data[['Age', 'Pclass']].apply(impute_age, axis=1)

Percentage of column ith missing 

**Remove Cabin as it contains many Nan Data**

In [ ]:
train_data.fillna(0,inplace=True)
test_data.fillna(0,inplace=True)

**Splitting labels and target**

In [ ]:
X = train_data.drop(columns=['Survived'])
Y = train_data['Survived']



#choose the features we want to train, just forget the float data
cate_features_index = np.where(X.dtypes != float)[0]


In [ ]:
from catboost import CatBoostClassifier, cv, Pool

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#make the x for train and test (also called validation data) 
xtrain,xtest,ytrain,ytest = train_test_split(X,Y,train_size=0.90,random_state=42)

In [ ]:
clf =CatBoostClassifier(eval_metric='Accuracy',use_best_model=True,random_seed=42)

**kfOLD validation**

In [ ]:
#now just to make the model to fit the data
clf.fit(xtrain,ytrain,cat_features=cate_features_index,eval_set=(xtest,ytest), early_stopping_rounds=50)

**Preprocessing Test_Data like train data**

In [ ]:
test_id = test_data.PassengerId

In [ ]:
test_data.isnull().sum()

In [ ]:
prediction = clf.predict(test_data)

In [ ]:
len(test_id),len(prediction)

In [ ]:
df_sub = pd.DataFrame()
df_sub['PassengerId'] = test_id
df_sub['Survived'] = prediction.astype(np.int)

df_sub.to_csv('gender_submission.csv', index=False)